# El Gran Dalmuti - QLearning

In [1]:
import numpy as np

### Basic game mechanics

In [2]:
NUM_CARD_VALUES = 13
JOKER = 12  # Jokers at index 12


def has_already_won(hand):
    """
    hand - vector with 13 entries (number of 1, 2, ..., 12, Jokers)
    """

    if len(hand.shape) == 1:
        return np.all(hand == 0)
    else:
        return np.all(hand == 0, axis=1)
    
    
def get_cards_array(card_type, num_cards):
    """ Vector representation of the cards of one kind. """
    
    cards_array = np.zeros(NUM_CARD_VALUES, dtype=np.int8)
    cards_array[card_type] = num_cards
    return cards_array
    

def possible_next_moves(hand, board):
    """
    Returns possible next moves as a list of tuples (new hand, new board)
    """
    
    card_type_in_board = np.argmax(board)
    num_cards_in_board = board[card_type_in_board] + board[JOKER]
    
    # You can always pass if it is not the initial move
    possible_hands = np.reshape(hand, (1, NUM_CARD_VALUES))
    possible_boards = np.reshape(board, (1, NUM_CARD_VALUES))
    
    if not has_already_won(hand):
        for card_type_in_hand in range(NUM_CARD_VALUES - 1, -1, -1):
            # You can play clean
            if card_type_in_hand < card_type_in_board and \
              hand[card_type_in_hand] >= num_cards_in_board:
                new_board = get_cards_array(card_type_in_hand, num_cards_in_board)
                new_hand = hand - new_board
                possible_hands = np.vstack([possible_hands, new_hand])
                possible_boards = np.vstack([possible_boards, new_board])

            # Or you can play dirty (with Joker(s))
            if card_type_in_hand != JOKER and hand[JOKER] > 0 and \
              card_type_in_hand < card_type_in_board and hand[card_type_in_hand] > 0 and \
              hand[card_type_in_hand] + hand[JOKER] >= num_cards_in_board:
                # Use one joker
                if hand[card_type_in_hand] + 1 >= num_cards_in_board:
                    joker_vec = get_cards_array(JOKER, 1)
                    new_board = get_cards_array(card_type_in_hand, num_cards_in_board - 1) + joker_vec
                    new_hand = hand - new_board
                    possible_hands = np.vstack([possible_hands, new_hand])
                    possible_boards = np.vstack([possible_boards, new_board])

                # Use two jokers
                if hand[JOKER] == 2 and num_cards_in_board > 2:
                    joker_vec = get_cards_array(JOKER, 2)
                    new_board = get_cards_array(card_type_in_hand, num_cards_in_board - 2) + joker_vec
                    new_hand = hand - new_board
                    possible_hands = np.vstack([possible_hands, new_hand])
                    possible_boards = np.vstack([possible_boards, new_board])
            
    return possible_hands, possible_boards
        
    
# Some tests
assert has_already_won(np.zeros(NUM_CARD_VALUES))
assert not has_already_won(np.ones(NUM_CARD_VALUES))
assert np.all(has_already_won(np.zeros((2, NUM_CARD_VALUES))) == np.array([True, True]))
assert np.all(has_already_won(np.array([[0,0,0,0,1], [0,0,0,0,0], [1,0,0,0,0]])) == np.array([False, True, False]))

assert np.all(get_cards_array(1, 2) == np.array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
assert np.all(get_cards_array(4, 3) == np.array([0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]))
assert not np.all(get_cards_array(4, 3) == np.array([0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]))

# Tests for possible moves
h, b = possible_next_moves(np.array([0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]))
assert np.all(h == np.array([[0., 2., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 1., 0., 2., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 2.],
                             [0., 1., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 1.]]))
assert np.all(b == np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]))
h, b = possible_next_moves(np.array([1, 2, 3, 1, 0, 0, 0, 3, 0, 4, 0, 0, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1]))
assert np.all(h == np.array([[1., 2., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 2.],
                             [1., 2., 3., 1., 0., 0., 0., 0., 0., 4., 0., 0., 2.],
                             [1., 2., 3., 1., 0., 0., 0., 1., 0., 4., 0., 0., 1.],
                             [1., 2., 3., 1., 0., 0., 0., 2., 0., 4., 0., 0., 0.],
                             [1., 2., 3., 0., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [1., 2., 0., 1., 0., 0., 0., 3., 0., 4., 0., 0., 2.],
                             [1., 2., 1., 1., 0., 0., 0., 3., 0., 4., 0., 0., 1.],
                             [1., 2., 2., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [1., 0., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 1.],
                             [1., 1., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [0., 2., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.]]))
assert np.all(b == np.array([[0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2.],
                             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                             [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
                             [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
                             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]]))

In [3]:
h, b = possible_next_moves(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]))
len(h)

31

### Generate Random Initial Game States

In [4]:
NUM_PLAYERS = 4
AVAILABLE_CARDS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]
PLAYER = list(range(NUM_PLAYERS))


def random_initial_cards():
    """ Random initial state for the game. """
    
    deck = np.array([], dtype=np.int8)

    for card_type in range(NUM_CARD_VALUES):
        deck = np.append(deck, np.array([card_type for _ in range(AVAILABLE_CARDS[card_type])]))
    
    np.random.shuffle(deck)
    
    chunk = deck.shape[0] // NUM_PLAYERS
    remainder = deck.shape[0] % NUM_PLAYERS
    first_player_initialized = False
    
    for playerIndex in range(NUM_PLAYERS):
        beginOfChunk = playerIndex * chunk + min(playerIndex, remainder)
        endOfChunk = (playerIndex + 1) * chunk + min(playerIndex + 1, remainder)
        player = np.zeros(NUM_CARD_VALUES, dtype=np.int8)
        
        for card in deck[beginOfChunk:endOfChunk]:
            player += get_cards_array(card, 1)
            
        if first_player_initialized:
            player_initial_hands = np.vstack([player_initial_hands, player])
        else:
            first_player_initialized = True
            player_initial_hands = player
        
    return player_initial_hands
        
        
random_initial_cards()

array([[0, 1, 1, 2, 3, 2, 0, 3, 2, 1, 3, 1, 1],
       [0, 1, 0, 0, 1, 2, 1, 2, 3, 4, 2, 4, 0],
       [1, 0, 1, 1, 0, 1, 3, 1, 1, 4, 1, 5, 1],
       [0, 0, 1, 1, 1, 1, 3, 2, 3, 1, 5, 2, 0]], dtype=int8)

In [8]:
np.sum(random_initial_cards(), axis=1) == 20 * np.ones(4)

array([ True,  True,  True,  True])

In [9]:
np.all(np.sum(random_initial_cards(), axis=1) == 20 * np.ones(NUM_PLAYERS))

True

### Encode Cards Vector for Indexing and Mapping

In [ ]:
card_bit_offset = [
      0,  2,  6, 10, 14, 18,  22,  25,  28,  31,  34,  36,  38,
     39, 41, 45, 49, 53, 57,  61,  64,  67,  70,  73,  75,  77,
     78, 80, 84, 88, 92, 96, 100, 103, 106, 109, 112, 114, 116,
    117  # sentinel
]

def encode_card_array(card_array):
    """
    Encodes a card vector into an int64.
    | Board | Hand | Already played |
     38   26 25  13 12             0
    """
    
    pass


def decode_card_repr(card_repr):
    """
    Decodes an int64 back to a card vector.
    """
    
    pass



### Q-Learning

In [ ]:
# Q-Learning
alpha = 0.8
gamma = 1.0
epsilon = 0.1

debug = False

# For plotting metrics
all_epochs = []
all_wins = []

player1States = np.array([np.zeros(9, dtype=int)])
player1QTable = np.array([np.zeros(9)])
player1 = 1

player2States = np.array([np.zeros(9, dtype=int)])
player2QTable = np.array([np.zeros(9)])
player2 = -1

def step(player, player_states, player_table, player_state, iterations=0, best=False):
    if debug:
        print("Player", player, "- State", player_state)
    
    # Possible actions
    actions_possible = (player_state == 0)
    
    # Exit if board full
    if not np.any(actions_possible):
        return True, player_states, player_table, player_state
    
    # Retrieve Q-Table for current state
    learned_values = player_table[np.all(player_states == player_state, axis=1)]

    # Add new Q-Table entry if necessary
    if learned_values.size > 0:
        learned_values = learned_values[0]
    else:
        learned_values = np.zeros(9, dtype=int)
        player_states = np.vstack((player_states, np.reshape(player_state, (1,9))))
        player_table = np.vstack((player_table, np.reshape(learned_values, (1,9))))
    
    if debug:
        print("Player", player, "- Learned Values", learned_values)
    
    # Decrease random decisions with increasing knowledge
    if not best and np.random.uniform() < (epsilon / min(1, (iterations / 20000))):
        # Random choice among possible actions
        action = np.random.choice(np.nonzero(actions_possible)[0])
    else:
        # Get best action
        possible_qvalues = np.where(actions_possible, learned_values, np.nan)
        action = np.random.choice(np.flatnonzero(np.isclose(
            possible_qvalues, np.nanmax(possible_qvalues))))
        
    # Next state
    next_state = np.where(np.arange(9) == action, player, player_state)
    
    # Old value
    old_value = learned_values[action]
    
    # Next maximum value
    if debug:
        print("Player", player, "- Player States", player_states)
        print("Player", player, "- Next State", next_state)
        print("Player", player, "- QTable", player_table)
    actions_possible_in_next_state = (next_state == 0)
    next_qvalues = player_table[np.all(player_states == next_state, axis=1)]
    next_max = np.nanmax(np.where(actions_possible_in_next_state, next_qvalues[0], np.nan)) \
        if next_qvalues.size > 0 else 0

    # Determine reward
    if almost_complete(next_state, -player):
        reward_earned = -1  # Penalty if other player can win game next turn
    elif reward(next_state, player):
        reward_earned = 1
    else:
        reward_earned = 0
    
    # Determine new value
    new_value = (1 - alpha) * old_value + alpha * (reward_earned + gamma * next_max)
    learned_values[action] = new_value
    player_table[np.all(player_states == player_state, axis=1)] = learned_values
    
    if debug:
        print("Player", player, "- QTable updated", player_table)
    
    # Return next state
    return (reward_earned == 1), player_states, player_table, next_state
    

last_iteration = 100001
    
for i in range(1, last_iteration + 1):
    epochs = 0
    
    current_state = np.zeros(9, dtype=int)
    
    while True:
        done, player1States, player1QTable, current_state = step(
            player1, player1States, player1QTable, current_state, i)
        
        if i == last_iteration:
            print("Player 1")
            print_board(current_state)
        
        if done:
            break
        
        done, player2States, player2QTable, current_state = step(
            player2, player2States, player2QTable, current_state, i)
        
        if i == last_iteration:
            print("Player 2")
            print_board(current_state)
        
        if done:
            break
        
        epochs += 1
            
    all_epochs.append(epochs)
    all_wins.append(current_state)
        
    if i % 100 == 0:
        clear_output(wait=True)
        print("Episode:", i)

In [5]:
a = { np.array([1,0,2,12]): (12, np.array([1,0,2,12])), np.array([1,0,1,12]): (10, np.array([1,0,1,12])) }

TypeError: unhashable type: 'numpy.ndarray'

In [23]:
import math
(3 ** 117), (2 ** 64), (3 ** 117) < (2 ** 64)

(66555937033867822607895549241096482953017615834735226163,
 18446744073709551616,
 False)

In [22]:
np.int64(3 ** 39)

4052555153018976267